In [12]:
import os

In [13]:
%pwd

'/Users/abdulrasheed/Desktop/ImageClassification'

In [14]:
os.chdir('../')

In [ ]:
# srouce/CNNClassifier/entity/config_entity.py

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path

In [ ]:
# source/CNNClassfier/config/configuration.py

from source.CNNClassifier.constant.__init_ import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from source.CNNClassifier.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, # stores cofig file path 'config/config.yaml'
        params_filepath = PARAMS_FILE_PATH): # stores params file path 'params.yaml'

        self.config = read_yaml(config_filepath) # reads the config.yaml file
        self.params = read_yaml(params_filepath) # reads the params.yaml file

        create_directories([self.config.artifacts_root]) # creates artifacts folder 'artifacts_root : artifacts'

    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion # config stores artifacts/data_ingestion
        # Accesses the data_ingestion section from the loaded configuration (self.config)
        # This section comes from a YAML file (parsed earlier by the read_yaml function) and contains all settings related to data ingestion.


        create_directories([config.root_dir]) # creates data_ingestion folder (root_dir : artifacts/data_ingestion)

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,              # artifacts/data_ingestion
            source_URL = config.source_URL,          #'https://github.com/AbdulRasheed1011/ImagesData/blob/main/Images.zip'
            local_data_file= config.local_data_file, # artifacts/data_ingestion/data.zip
            unzip_dir = config.unzip_dir             # artifacts/data_ingestion
        )

        return data_ingestion_config             


In [ ]:
### Components data_ingestion.py

In [19]:
import os
import urllib.request as request
import zipfile  
from source.CNNClassifier import logger
from source.CNNClassifier.utils.common import get_size

In [20]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, header = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file

            )
            logger.info(f"{filename} download! with following info: \n{header}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        extract zip file to data diretory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        print(f"File path: {self.config.local_data_file}")

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}") 


In [21]:
### pipeline

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    logger.error(f"Error during data ingestion: {str(e)}")

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'